# Medical Machine Translation Dataset Generation (Chinese → Thai)
## Google Colab Version with Open-Source Model

This notebook adapts the original `generate_dataset.py` script to run in Google Colab and uses an open-source translation model (`Helsinki-NLP/opus-mt-zh-th`) from Hugging Face instead of the DeepSeek API.

**Instructions:**
1. Run the **Install Dependencies** cell.
2. Run the **Imports** cell.
3. Run the **Dataset Templates** cell.
4. Run the **Hugging Face Model Setup** cell. This will download the model (may take a few minutes).
5. Run the **Helper Functions** cells (Translation, Logging, File Utilities).
6. Run the **Core Data Generation Logic** cell.
7. Run the **Main Script Utilities** cell.
8. Configure parameters in the **Colab Parameter Setup and Execution** cell.
9. Run the **Colab Parameter Setup and Execution** cell to generate the dataset.

In [ ]:
# Cell 1: Install Dependencies
!pip install pandas transformers torch sentencepiece tqdm accelerate

In [ ]:
# Cell 2: Imports
import argparse
import pandas as pd
import random
import os
import sys
import time
from tqdm.auto import tqdm  # Use tqdm.auto for notebook compatibility
import datetime
import string
import random as pyrandom
import logging

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

In [ ]:
# Cell 3: Dataset Templates
# (Content from dataset_templates.py)

MEDICAL_TOPICS = [
    {"zh": "感冒", "th": "ไข้หวัด", "desc": "普通感冒症状与治疗"},
    {"zh": "高血压", "th": "ความดันโลหิตสูง", "desc": "高血压管理与药物"},
    {"zh": "糖尿病", "th": "โรคเบาหวาน", "desc": "糖尿病护理与饮食建议"},
    {"zh": "胃痛", "th": "อาการปวดท้อง", "desc": "胃部不适与消化问题"},
    {"zh": "头痛", "th": "อาการปวดศีรษะ", "desc": "各种类型的头痛及其原因"},
    {"zh": "皮肤过敏", "th": "โรคภูมิแพ้ผิวหนัง", "desc": "皮肤过敏症状与治疗"},
    {"zh": "失眠", "th": "อาการนอนไม่หลับ", "desc": "失眠问题与改善方法"},
    {"zh": "关节炎", "th": "โรคข้ออักเสบ", "desc": "关节炎疼痛与护理"},
    {"zh": "哮喘", "th": "โรคหอบหืด", "desc": "哮喘发作与预防"},
    {"zh": "抑郁症", "th": "โรคซึมเศร้า", "desc": "抑郁症的心理支持与治疗"},
    {"zh": "骨折", "th": "กระดูกหัก", "desc": "骨折处理与康复"},
    {"zh": "肺炎", "th": "โรคปอดบวม", "desc": "肺炎症状与治疗方法"},
    {"zh": "心脏病", "th": "โรคหัวใจ", "desc": "心脏健康与预防"},
    {"zh": "肾结石", "th": "โรคนิ่วในไต", "desc": "肾结石的症状与处理"},
    {"zh": "中风", "th": "โรคหลอดเลือดสมอง", "desc": "中风的急救与康复"},
    {"zh": "视力模糊", "th": "อาการตามัว", "desc": "视力问题与眼科检查"},
    {"zh": "牙痛", "th": "อาการปวดฟัน", "desc": "牙痛原因与牙科治疗"},
    {"zh": "过敏性鼻炎", "th": "โรคจมูกอักเสบจากภูมิแพ้", "desc": "过敏性鼻炎的控制与治疗"},
    {"zh": "消化不良", "th": "อาหารไม่ย่อย", "desc": "消化不良的症状与饮食调整"},
    {"zh": "烧伤", "th": "แผลไฟไหม้น้ำร้อนลวก", "desc": "烧伤处理与皮肤护理"}
]

DIALOGUE_SAMPLES = [
    {"context": "患者主诉发热、咳嗽三天。", "symptom": "发热, 咳嗽", "source": "医生，我这几天感觉很不舒服，发烧还咳嗽，有三天了。体温量了吗？最高多少度？", "target": "คุณหมอคะ mấy วันนี้รู้สึกไม่สบายเลยค่ะ เป็นไข้แล้วก็ไอด้วย เป็นมา 3 วันแล้วค่ะ วัดไข้ไหมคะ สูงสุดเท่าไหร่คะ?"},
    {"context": "患者咨询高血压药物的副作用。", "symptom": "高血压, 药物副作用", "source": "我一直在吃降压药，最近感觉有些头晕乏力，是药物的副作用吗？您吃的具体是什么药？这种药常见的副作用确实包括头晕。", "target": "ทานยาลดความดันมาตลอด ช่วงนี้รู้สึกเวียนหัว อ่อนเพลีย เป็นผลข้างเคียงของยาหรือเปล่าครับ? คุณทานยาอะไรอยู่ครับ? ยาตัวนี้ผลข้างเคียงที่พบบ่อยก็มีอาการเวียนหัวครับ"},
    {"context": "患者血糖控制不佳，医生调整用药。", "symptom": "糖尿病, 血糖高", "source": "我最近测血糖还是偏高，饮食也注意了，是不是药效不够？根据您的情况，我们可能需要调整一下胰岛素的剂量。", "target": "ช่วงนี้วัดน้ำตาลในเลือดก็ยังสูงอยู่ค่ะ อาหารก็ระวังแล้ว เป็นเพราะยาไม่พอหรือเปล่าคะ? จากอาการของคุณ เราอาจจะต้องปรับขนาดยาอินซูลินหน่อยนะคะ"},
    {"context": "患者胃痛，医生询问饮食习惯。", "symptom": "胃痛, 饮食不规律", "source": "医生，我胃痛好几天了，特别是饭后。您平时吃饭规律吗？有没有吃什么刺激性的食物？", "target": "คุณหมอครับ ผมปวดท้องมาหลายวันแล้ว โดยเฉพาะหลังกินข้าวครับ ปกติคุณทานข้าวตรงเวลามั้ยครับ? ได้ทานอาหารรสจัดอะไรบ้างมั้ยครับ?"},
    {"context": "患者因偏头痛就诊。", "symptom": "偏头痛", "source": "我头痛得厉害，感觉是偏头痛又犯了。您描述一下疼痛的性质和位置好吗？", "target": "ปวดหัวมากเลยค่ะ รู้สึกเหมือนไมเกรนกำเริบอีกแล้ว ช่วยอธิบายลักษณะอาการปวดและตำแหน่งได้ไหมคะ?"},
    {"context": "儿童皮肤出现红疹，怀疑过敏。", "symptom": "皮肤红疹, 过敏", "source": "医生您看，我家孩子身上起了好多红疹子，痒得不行，是不是过敏了？最近有没有接触什么新的东西或者吃什么特别的食物？", "target": "คุณหมอดูสิคะ ลูกฉันมีผื่นแดงขึ้นเต็มตัวเลย คันมาก ไม่รู้ว่าแพ้อะไรรึเปล่าคะ? ช่วงนี้ได้สัมผัสอะไรใหม่ๆ หรือกินอาหารอะไรเป็นพิเศษไหมคะ?"},
    {"context": "患者长期失眠，寻求帮助。", "symptom": "失眠", "source": "我最近总是失眠，晚上翻来覆去睡不着，白天没精神。这种情况持续多久了？有没有什么压力或者心事？", "target": "ช่วงนี้ฉันนอนไม่หลับตลอดเลยค่ะ กลางคืนพลิกตัวไปมานอนไม่หลับ กลางวันก็ไม่มีแรงเลยค่ะ เป็นแบบนี้มานานแค่ไหนแล้วคะ? มีเรื่องเครียดหรือกังวลอะไรไหมคะ?"},
    {"context": "老年患者关节疼痛，咨询治疗方案。", "symptom": "关节炎, 关节痛", "source": "医生，我的膝盖关节一到阴雨天就疼得厉害，有什么好办法缓解吗？您这种情况可能是关节炎，我们可以先做个检查确诊一下。", "target": "คุณหมอคะ ข้อเข่าของฉันพอถึงวันฟ้าครึ้มฝนตกทีไรก็ปวดมากเลยค่ะ มีวิธีบรรเทาดีๆ บ้างไหมคะ? อาการแบบนี้น่าจะเป็นข้ออักเสบนะคะ เราลองตรวจดูก่อนเพื่อยืนยันนะคะ"},
    {"context": "患者哮喘发作，呼吸困难。", "symptom": "哮喘, 呼吸困难", "source": "我哮喘又犯了，感觉喘不上气来。赶紧用一下您的急救喷雾，我给您听一下肺部。", "target": "โรคหอบหืดของฉันกำเริบอีกแล้วค่ะ รู้สึกหายใจไม่ออกเลย รีบใช้ยาพ่นฉุกเฉินของคุณก่อนนะคะ เดี๋ยวหมอขอฟังเสียงปอดหน่อยค่ะ"},
    {"context": "患者情绪低落，怀疑抑郁。", "symptom": "情绪低落, 抑郁倾向", "source": "我最近心情一直很差，对什么都提不起兴趣，是不是有点抑郁了？能具体说说您的感受和这种情况持续的时间吗？", "target": "ช่วงนี้อารมณ์ไม่ดีเลยค่ะ ไม่สนใจอะไรเลย ไม่รู้ว่าเป็นซึมเศร้ารึเปล่าคะ? ช่วยเล่าความรู้สึกของคุณกับระยะเวลาที่เป็นแบบนี้ให้ฟังหน่อยได้ไหมคะ?"}    
]

QA_SAMPLES = [
    {"topic": "感冒", "context": "关于普通感冒的常见问题。", "question": "感冒一般几天能好？", "answer": "普通感冒通常持续7到10天，但有些症状如咳嗽可能会持续更长时间。", "q_th": "ไข้หวัดธรรมดากี่วันถึงจะหาย?", "a_th": "ไข้หวัดธรรมดามักจะหายภายใน 7 ถึง 10 วัน แต่อาการบางอย่างเช่น อาจจะยังคงอยู่นานกว่านั้น"},
    {"topic": "高血压", "context": "高血压患者的饮食注意事项。", "question": "高血压患者饮食上要注意什么？", "answer": "高血压患者应注意低盐、低脂饮食，多吃蔬菜水果，控制体重。", "q_th": "ผู้ป่วยความดันโลหิตสูงควรระวังอะไรในการรับประทานอาหาร?", "a_th": "ผู้ป่วยความดันโลหิตสูงควรรับประทานอาหารที่มีเกลือต่ำ ไขมันต่ำ รับประทานผักและผลไม้ให้มาก และควบคุมน้ำหนักตัว"},
    {"topic": "糖尿病", "context": "糖尿病的典型症状。", "question": "糖尿病有哪些典型症状？", "answer": "糖尿病的典型症状包括多饮、多尿、多食和体重减轻，即“三多一少”。", "q_th": "โรคเบาหวานมีอาการ典型อย่างไรบ้าง?", "a_th": "อาการทั่วไปของโรคเบาหวาน ได้แก่ ดื่มน้ำมาก ปัสสาวะบ่อย กินจุ และน้ำหนักลด หรือที่เรียกว่า “สามมากหนึ่งน้อย”"},
    {"topic": "胃痛", "context": "胃痛的可能原因。", "question": "什么原因可能导致胃痛？", "answer": "胃痛的可能原因很多，包括消化不良、胃炎、胃溃疡、胃食管反流等。如果胃痛持续或严重，建议就医检查。", "q_th": "สาเหตุใดบ้างที่อาจทำให้ปวดท้อง?", "a_th": "สาเหตุของอาการปวดท้องมีได้หลายอย่าง เช่น อาหารไม่ย่อย กระเพาะอาหารอักเสบ แผลในกระเพาะอาหาร กรดไหลย้อน หากปวดท้องต่อเนื่องหรือรุนแรง ควรไปพบแพทย์เพื่อตรวจ"},
    {"topic": "头痛", "context": "缓解头痛的方法。", "question": "如何缓解头痛？", "answer": "缓解头痛的方法因人而异，可以尝试休息、按摩、冷敷或热敷、饮用足够的水。如果头痛频繁或严重，应咨询医生。", "q_th": "จะบรรเทาอาการปวดหัวได้อย่างไร?", "a_th": "วิธีบรรเทาอาการปวดหัวแตกต่างกันไปในแต่ละบุคคล ลองพักผ่อน นวด ประคบเย็นหรือประคบร้อน ดื่มน้ำให้เพียงพอ หากปวดหัวบ่อยหรือรุนแรง ควรปรึกษาแพทย์"}
]

MEDICAL_REASONING_SAMPLES = [
    {"context": "患者，男，65岁，有长期吸烟史，近期出现持续性咳嗽、咳痰带血、胸痛和体重下降。胸部X光片显示肺部有占位性病变。", "question": "根据患者的病史、症状和影像学检查结果，最可能的诊断是什么？为什么？", "answer": "最可能的诊断是肺癌。原因如下：1. 患者为老年男性，有长期吸烟史，是肺癌的高危人群。2. 患者的症状如持续性咳嗽、咳痰带血、胸痛和体重下降是肺癌的常见临床表现。3. 胸部X光片显示肺部占位性病变，高度提示恶性肿瘤的可能。", "q_th": "จากประวัติผู้ป่วย อาการ และผลการตรวจทางรังสีวิทยา การวินิจฉัยที่เป็นไปได้มากที่สุดคืออะไร เพราะเหตุใด?", "a_th": "การวินิจฉัยที่เป็นไปได้มากที่สุดคือมะเร็งปอด ด้วยเหตุผลดังต่อไปนี้: 1. ผู้ป่วยเป็นชายสูงอายุ มีประวัติสูบบุหรี่มานาน ซึ่งเป็นกลุ่มเสี่ยงสูงของมะเร็งปอด 2. อาการของผู้ป่วย เช่น ไอเรื้อรัง เสมหะปนเลือด เจ็บหน้าอก และน้ำหนักลด เป็นอาการทางคลินิกที่พบบ่อยของมะเร็งปอด 3. ภาพถ่ายรังสีทรวงอกพบก้อนในปอด ซึ่งบ่งชี้อย่างมากถึงความเป็นไปได้ของเนื้องอกร้ายแรง"},
    {"context": "患者，女，30岁，因突发右下腹剧烈疼痛伴恶心、呕吐、发热就诊。体格检查发现右下腹压痛、反跳痛明显，麦氏点压痛阳性。实验室检查白细胞计数及中性粒细胞比例显著升高。", "question": "结合患者的临床表现和实验室检查结果，应首先考虑哪种急腹症？主要的诊断依据是什么？", "answer": "应首先考虑急性阑尾炎。主要诊断依据：1. 典型的转移性右下腹疼痛，伴有恶心、呕吐、发热等症状。2. 体格检查发现右下腹固定压痛、反跳痛，麦氏点压痛阳性，这些是急性阑尾炎的典型体征。3. 实验室检查显示白细胞计数和中性粒细胞比例升高，提示存在急性细菌感染。", "q_th": "จากอาการทางคลินิกและผลการตรวจทางห้องปฏิบัติการของผู้ป่วย ควรพิจารณาภาวะฉุกเฉินในช่องท้องชนิดใดเป็นอันดับแรก และหลักฐานสำคัญในการวินิจฉัยคืออะไร?", "a_th": "ควรพิจารณาไส้ติ่งอักเสบเฉียบพลันเป็นอันดับแรก หลักฐานสำคัญในการวินิจฉัยคือ: 1. อาการปวดท้องด้านขวาล่างแบบย้ายที่อย่างเฉียบพลัน ร่วมกับอาการคลื่นไส้ อาเจียน และมีไข้ 2. การตรวจร่างกายพบอาการกดเจ็บและปล่อยเจ็บที่ท้องด้านขวาล่างชัดเจน และกดเจ็บที่จุดแมคเบอร์นีย์ ซึ่งเป็นอาการแสดงที่典型ของไส้ติ่งอักเสบเฉียบพลัน 3. ผลการตรวจทางห้องปฏิบัติการพบจำนวนเม็ดเลือดขาวและสัดส่วนนิวโทรฟิลสูงขึ้น ซึ่งบ่งชี้ว่ามีการติดเชื้อแบคทีเรียเฉียบพลัน"}
]

SUMMARIZATION_SAMPLES = [
    {"context": "患者：医生，我最近总是感觉很累，头也晕晕的，有时候还会心慌。我今年55岁，有高血压病史5年了，一直在吃药控制，但最近感觉效果不太好。医生：您好，您这种情况多久了？除了这些症状，还有没有胸闷、气短的感觉？血压最近测过吗？规律吗？患者：大概有半个多月了吧。有时候会觉得胸口闷闷的，上楼梯会有点喘。血压我每天都测，早上比较高，大概在150/95mmHg左右，下午会好一点。医生：嗯，您这种情况需要重视。高血压控制不佳，加上您描述的症状，我们需要排除一下心脏方面的问题。我建议您先做一个心电图和心脏彩超检查，看看心脏结构和功能有没有异常。另外，我们可能需要调整一下您的降压药物。您目前在服用什么药物？剂量是多少？患者：我现在吃的是硝苯地平缓释片，一天一次，一次一片。医生：好的，了解了。我们先做检查，根据检查结果再制定下一步的治疗方案。平时饮食注意低盐低脂，保持情绪稳定，适当活动，但不要剧烈运动。", "summary_zh": "患者为55岁男性，有5年高血压病史，近期感乏力、头晕、心慌半月余，伴胸闷、活动后气短。晨起血压较高（150/95mmHg）。目前服用硝苯地平缓释片控制血压。医生怀疑心脏问题，建议行心电图、心脏彩超检查，并可能调整降压药物。嘱患者低盐低脂饮食，稳定情绪，适当活动。", "summary_th": "ผู้ป่วยชายอายุ 55 ปี มีประวัติความดันโลหิตสูง 5 ปี ช่วงครึ่งเดือนที่ผ่านมารู้สึกอ่อนเพลีย เวียนศีรษะ ใจสั่น ร่วมกับแน่นหน้าอก หายใจลำบากเมื่อออกแรง ความดันโลหิตตอนเช้าค่อนข้างสูง (150/95 mmHg) ปัจจุบันรับประทานยา Nifedipine SR เพื่อควบคุมความดัน แพทย์สงสัยปัญหาเกี่ยวกับหัวใจ แนะนำให้ตรวจคลื่นไฟฟ้าหัวใจและอัลตราซาวนด์หัวใจ และอาจปรับยา ลดความดันโลหิต แนะนำให้ผู้ป่วยรับประทานอาหารเค็มน้อยไขมันต่ำ ควบคุมอารมณ์ และออกกำลังกายพอประมาณ"},
    {"context": "患者：医生，我孩子最近老是咳嗽，晚上尤其厉害，有时候咳得都睡不好觉。也没发烧，就是咳。医生：您好，孩子这种情况多久了？咳嗽有痰吗？是什么颜色的痰？有没有鼻塞流涕或者其他不舒服？患者：大概一周多了。开始是干咳，这两天有点痰，白色的黏痰。鼻子好像有点塞，偶尔打喷嚏。精神状态还可以，就是玩的时候容易咳。医生：好的。根据您描述的情况，孩子可能是上呼吸道感染或者支气管炎初期。晚上咳嗽加重，白色黏痰，需要注意一下。我建议先给孩子查一个血常规，看看有没有细菌感染的指征。另外，可以做个雾化治疗帮助化痰止咳。饮食上注意清淡，多喝水，避免去人多的地方，以免交叉感染。", "summary_zh": "患儿咳嗽一周余，夜间为甚，初为干咳，后有白色黏痰，伴鼻塞、偶有喷嚏，无发热。医生初步考虑上呼吸道感染或支气管炎初期，建议查血常规，行雾化治疗，并嘱清淡饮食、多饮水、避免交叉感染。", "summary_th": "เด็กมีอาการไอมาประมาณ 1 สัปดาห์ ไอมากโดยเฉพาะตอนกลางคืน เริ่มแรกเป็นอาการไอแห้งๆ ต่อมามีเสมหะสีขาวข้น ร่วมกับอาการคัดจมูก จามบ้างเป็นครั้งคราว ไม่มีไข้ แพทย์เบื้องต้นสันนิษฐานว่าเป็นการติดเชื้อทางเดินหายใจส่วนบนหรือหลอดลมอักเสบระยะแรก แนะนำให้ตรวจนับเม็ดเลือด ทำการพ่นยา และแนะนำให้รับประทานอาหารรสอ่อน ดื่มน้ำมากๆ และหลีกเลี่ยงการติดเชื้อซ้ำซ้อน"}
]

# Note: Prompt templates from the original script are not directly used for the Helsinki-NLP model,
# as it's a direct translation model. We will feed the source Chinese text directly.

In [ ]:
# Cell 4: Hugging Face Model Setup
MODEL_NAME = "Helsinki-NLP/opus-mt-zh-th"
translator = None

def initialize_translator():
    global translator
    if translator is None:
        print(f"[INFO] Initializing translator with model: {MODEL_NAME}...")
        try:
            # Check for GPU availability
            device = 0 if torch.cuda.is_available() else -1 # 0 for first GPU, -1 for CPU
            if device == 0:
                print("[INFO] CUDA (GPU) is available. Using GPU for translation.")
            else:
                print("[INFO] CUDA (GPU) not available. Using CPU for translation.")
            
            translator = pipeline("translation", model=MODEL_NAME, device=device)
            print("[INFO] Translator initialized successfully.")
        except Exception as e:
            print(f"[ERROR] Failed to initialize translator: {e}")
            print("[INFO] Falling back to CPU if GPU initialization failed.")
            try:
                translator = pipeline("translation", model=MODEL_NAME, device=-1)
                print("[INFO] Translator initialized successfully on CPU.")
            except Exception as e2:
                print(f"[ERROR] Failed to initialize translator on CPU: {e2}")
                translator = None # Ensure translator is None if all attempts fail
    return translator

# Call initialize_translator() once to load the model when the cell is run.
# Subsequent calls will use the already loaded model.
translator = initialize_translator() 

In [ ]:
# Cell 5: Helper function for translation
def translate_text_hf(text_list, current_translator):
    if not current_translator:
        log_error("TranslationError", -1, "Translator not initialized.")
        if isinstance(text_list, list):
            return ["Translation unavailable" for _ in text_list]
        else:
            return "Translation unavailable"
    try:
        # The pipeline can handle single string or list of strings
        if isinstance(text_list, str): # single text
             # Helsinki models expect src_lang and tgt_lang to be part of model name or config
             # For opus-mt-zh-th, it's implicitly Chinese to Thai.
            translation_result = current_translator(text_list)
            return translation_result[0]['translation_text']
        elif isinstance(text_list, list): # batch of texts
            # Process in batches if the list is very large to avoid OOM, though pipeline handles some batching.
            # For simplicity here, we pass the whole list. Consider actual batching for very large inputs.
            translation_results = current_translator(text_list, batch_size=8) # Adjust batch_size as needed
            return [res['translation_text'] for res in translation_results]
    except Exception as e:
        log_error("TranslationError", -1, f"Hugging Face API error: {e}")
        if isinstance(text_list, list):
            return [f"Error: {e}" for _ in text_list]
        else:
            return f"Error: {e}"

In [ ]:
# Cell 6: Logging and Metadata functions
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, f"generate_dataset_colab_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    filename=LOG_FILE,
    filemode='a',
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)

def log_error(error_type, sample_idx, message):
    logging.error(f"{error_type} | Sample: {sample_idx} | {message}")
    print(f"[ERRORLOG] {error_type} | Sample: {sample_idx} | {message}") # also print to console for Colab visibility

def add_metadata(df):
    """Add metadata to the dataset"""
    metadata = {
        "generator": "HuggingFace (Helsinki-NLP/opus-mt-zh-th)",
        "model": MODEL_NAME,
        "license": "CC BY-SA-NC 4.0 (Dataset) / Apache 2.0 (Model)",
        "generation_date": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "version": "2.0.0_colab_hf",
    }
    for key, value in metadata.items():
        df[f"_metadata_{key}"] = value
    return df

In [ ]:
# Cell 7: File and Path Utilities
def random_id(prefix="D"):
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    rand = ''.join(pyrandom.choices(string.ascii_uppercase + string.digits, k=4))
    return f"{prefix}{rand}_{ts}"

def get_datasets_dir():
    """Return the datasets directory, create if not exists (Colab: /content/datasets/)."""
    # In Colab, os.getcwd() is /content
    datasets_dir = os.path.join(os.getcwd(), "datasets")
    os.makedirs(datasets_dir, exist_ok=True)
    return datasets_dir

def unique_dataset_filename(base, ext, prefix="D"):
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    rand = ''.join(pyrandom.choices(string.ascii_uppercase + string.digits, k=4))
    return f"{base}_{prefix}{rand}_{ts}{ext}"

def save_dataset(df, output, file_format):
    if not output:
        datasets_dir = get_datasets_dir()
        base = "dataset"
        ext = "." + file_format
        filename = unique_dataset_filename(base, ext)
        output_path = os.path.join(datasets_dir, filename)
    else:
        # Ensure output path is absolute or relative to /content for Colab
        if not os.path.isabs(output) and not output.startswith("/content") :
            output = os.path.join("/content", output) # Default to /content if relative path given
        
        output_dir_user = os.path.dirname(output)
        if output_dir_user and not os.path.exists(output_dir_user):
            os.makedirs(output_dir_user, exist_ok=True) # Create user-specified directory
            
        if os.path.isdir(output):
            base = "dataset"
            ext = "." + file_format
            filename = unique_dataset_filename(base, ext)
            output_path = os.path.join(output, filename)
        else:
            base, ext_from_output = os.path.splitext(os.path.basename(output))
            if not base:
                base = "dataset"
            # Use specified extension if provided, otherwise from format arg
            ext = ext_from_output if ext_from_output else "." + file_format 
            prefix = base[0].upper() if base else "D"
            # Use the filename as is if it's a full path with extension
            if os.path.dirname(output) and ext_from_output : # User provided full path with extension
                 output_path = output
            else: # Auto generate filename or save in specified dir
                filename = unique_dataset_filename(base, ext, prefix)
                output_dir = os.path.dirname(output) or get_datasets_dir()
                os.makedirs(output_dir, exist_ok=True)
                output_path = os.path.join(output_dir, filename)

    print(f"[INFO] Attempting to save dataset to: {output_path}")
    if file_format == 'csv':
        df.to_csv(output_path, index=False)
    elif file_format == 'json':
        df.to_json(output_path, orient='records', force_ascii=False, indent=2)
    elif file_format == 'jsonl':
        df.to_json(output_path, orient='records', force_ascii=False, lines=True)
    elif file_format == 'txt':
        with open(output_path, 'w', encoding='utf-8') as f:
            for row in df.itertuples(index=False):
                f.write(str(row) + '
')
    elif file_format == 'arrow' or file_format == 'parquet': # Parquet is a common use for arrow tables
        try:
            import pyarrow as pa
            import pyarrow.parquet as pq
            table = pa.Table.from_pandas(df)
            pq.write_table(table, output_path)
        except ImportError:
            print("[ERROR] pyarrow library not found. Please install it to save in Arrow/Parquet format.")
            log_error("SaveDatasetError", -1, "pyarrow not installed for Arrow/Parquet format.")
            return # Exit if cannot save
    else:
        raise ValueError(f"Unsupported file format: {file_format}")
    print(f"[SUCCESS] Dataset saved: {output_path} ({len(df)} samples)")

In [ ]:
# Cell 8: Core Data Generation Logic

def _process_dialogue_sample_hf(args_tuple):
    i, topic, sample_data, current_translator, log_fn = args_tuple
    source_text = sample_data["source"]
    target_text = translate_text_hf(source_text, current_translator)
    
    if "Error:" in target_text or "Translation unavailable" in target_text:
        log_fn("TranslationFailure", i, f"Failed to translate. Source: {source_text}, Error: {target_text}")
        # Fallback to original sample target if provided, or use error message
        target_text = sample_data.get("target", target_text) 
        
    return {"context": sample_data["context"], "source": source_text, "target": target_text, "original_index": i}

def generate_dialogue_dataset_hf(n_samples=100, seed=42, current_translator=None):
    """Generate dialogue dataset using Hugging Face model"""
    if not current_translator:
        print("[ERROR] Translator not available for dialogue generation.")
        return pd.DataFrame()
        
    random.seed(seed)
    print(f"[INFO] Creating prompt combinations for {n_samples} dialogue samples...")
    
    all_prompt_combinations = []
    for topic in MEDICAL_TOPICS:
        for sample in DIALOGUE_SAMPLES:
            all_prompt_combinations.append((topic, sample))
    
    if not all_prompt_combinations:
        print("[ERROR] No dialogue samples or medical topics defined.")
        return pd.DataFrame()
        
    selected_combinations = []
    if n_samples <= len(all_prompt_combinations):
        selected_combinations = random.sample(all_prompt_combinations, n_samples)
    else:
        selected_combinations = all_prompt_combinations.copy()
        remaining = n_samples - len(all_prompt_combinations)
        combo_usage = {i: 1 for i in range(len(all_prompt_combinations))} 
        all_indices = list(range(len(all_prompt_combinations)))
        for _ in range(remaining):
            min_usage = min(combo_usage.values())
            least_used_indices = [idx for idx, count in combo_usage.items() if count == min_usage]
            selected_idx = random.choice(least_used_indices if least_used_indices else all_indices) # Fallback if all used equally
            selected_combinations.append(all_prompt_combinations[selected_idx])
            combo_usage[selected_idx] += 1
            
    tasks_args = []
    for i, (topic, sample) in enumerate(selected_combinations):
        tasks_args.append((i, topic, sample, current_translator, log_error))
    
    print(f"[INFO] Created {len(tasks_args)} dialogue tasks.")
    
    data = []
    # Process tasks (simple loop, consider batching for HF pipeline for better performance)
    # The HF pipeline itself can handle batching if a list of strings is passed.
    # Here, _process_dialogue_sample_hf translates one by one.
    source_texts_batch = [task_arg[2]["source"] for task_arg in tasks_args]
    translated_texts_batch = translate_text_hf(source_texts_batch, current_translator)
    
    for i, task_arg_tuple in enumerate(tqdm(tasks_args, total=len(tasks_args), desc="Generating Dialogue Samples")):
        idx, topic, sample_data, _, _ = task_arg_tuple
        target_text = translated_texts_batch[i]
        if "Error:" in target_text or "Translation unavailable" in target_text:
            log_error("TranslationFailure", idx, f"Failed to translate. Source: {sample_data["source"]}, Error: {target_text}")
            target_text = sample_data.get("target", target_text) 
        data.append({"context": sample_data["context"], "source": sample_data["source"], "target": target_text})
        
    df = pd.DataFrame(data)
    df = add_metadata(df)
    return df

def generate_qa_dataset_hf(n_samples=100, seed=42, current_translator=None):
    """Generate QA dataset using Hugging Face model"""
    if not current_translator:
        print("[ERROR] Translator not available for QA generation.")
        return pd.DataFrame()
        
    data = []
    random.seed(seed)
    
    if not QA_SAMPLES:
        print("[ERROR] No QA_SAMPLES defined.")
        return pd.DataFrame()
        
    for i in tqdm(range(n_samples), desc="Generating QA Samples")):
        sample = QA_SAMPLES[i % len(QA_SAMPLES)]
        question_zh = sample["question"]
        answer_zh = sample["answer"]
        
        try:
            translations = translate_text_hf([question_zh, answer_zh], current_translator)
            question_th = translations[0]
            answer_th = translations[1]
            
            if "Error:" in question_th or "Translation unavailable" in question_th:
                log_error("TranslationFailure", i, f"Q-Trans fail: {question_zh} -> {question_th}")
                question_th = sample.get("q_th", question_th) # Fallback
            if "Error:" in answer_th or "Translation unavailable" in answer_th:
                log_error("TranslationFailure", i, f"A-Trans fail: {answer_zh} -> {answer_th}")
                answer_th = sample.get("a_th", answer_th) # Fallback
                
        except Exception as e:
            log_error("HF_API_Error_QA", i, str(e))
            question_th = sample.get("q_th", f"Error: {e}")
            answer_th = sample.get("a_th", f"Error: {e}")
        
        data.append({
            "context": sample["context"],
            "question_zh": question_zh,
            "answer_zh": answer_zh,
            "question_th": question_th,
            "answer_th": answer_th
        })
        
    df = pd.DataFrame(data)
    df = add_metadata(df)
    return df

def generate_reasoning_dataset_hf(n_samples=100, seed=42, current_translator=None):
    """Generate medical reasoning QA dataset using Hugging Face model"""
    if not current_translator:
        print("[ERROR] Translator not available for reasoning generation.")
        return pd.DataFrame()
        
    data = []
    random.seed(seed)
    if not MEDICAL_REASONING_SAMPLES:
        print("[ERROR] No MEDICAL_REASONING_SAMPLES defined.")
        return pd.DataFrame()
        
    for i in tqdm(range(n_samples), desc="Generating Reasoning Samples")):
        sample = MEDICAL_REASONING_SAMPLES[i % len(MEDICAL_REASONING_SAMPLES)]
        question_zh = sample["question"]
        answer_zh = sample["answer"]
        try:
            translations = translate_text_hf([question_zh, answer_zh], current_translator)
            question_th = translations[0]
            answer_th = translations[1]
            
            if "Error:" in question_th or "Translation unavailable" in question_th:
                log_error("TranslationFailure", i, f"Q-Trans fail: {question_zh} -> {question_th}")
                question_th = sample.get("q_th", question_th)
            if "Error:" in answer_th or "Translation unavailable" in answer_th:
                log_error("TranslationFailure", i, f"A-Trans fail: {answer_zh} -> {answer_th}")
                answer_th = sample.get("a_th", answer_th)
                
        except Exception as e:
            log_error("HF_API_Error_Reasoning", i, str(e))
            question_th = sample.get("q_th", f"Error: {e}")
            answer_th = sample.get("a_th", f"Error: {e}")
            
        data.append({
            "context": sample["context"],
            "question_zh": question_zh,
            "answer_zh": answer_zh,
            "question_th": question_th,
            "answer_th": answer_th
        })
    df = pd.DataFrame(data)
    df = add_metadata(df)
    return df

def generate_summarization_dataset_hf(n_samples=100, seed=42, current_translator=None):
    """Generate summarization dataset by translating Chinese summaries using Hugging Face model"""
    if not current_translator:
        print("[ERROR] Translator not available for summarization generation.")
        return pd.DataFrame()
        
    data = []
    random.seed(seed)
    if not SUMMARIZATION_SAMPLES:
        print("[ERROR] No SUMMARIZATION_SAMPLES defined.")
        return pd.DataFrame()
        
    for i in tqdm(range(n_samples), desc="Generating Summarization Samples")):
        sample = SUMMARIZATION_SAMPLES[i % len(SUMMARIZATION_SAMPLES)]
        summary_zh = sample["summary_zh"]
        try:
            summary_th = translate_text_hf(summary_zh, current_translator)
            if "Error:" in summary_th or "Translation unavailable" in summary_th:
                log_error("TranslationFailure", i, f"Summary-Trans fail: {summary_zh} -> {summary_th}")
                summary_th = sample.get("summary_th", summary_th) # Fallback
        except Exception as e:
            log_error("HF_API_Error_Summarization", i, str(e))
            summary_th = sample.get("summary_th", f"Error: {e}")
            
        data.append({
            "context": sample["context"],
            "summary_zh": summary_zh,
            "summary_th": summary_th
        })
    df = pd.DataFrame(data)
    df = add_metadata(df)
    return df

In [ ]:
# Cell 9: Main Script Utilities
def print_ascii_banner():
    banner = r'''
    ▒███████▒ ▒█████   ███▄ ▄███▓ ▄▄▄▄    ██▓▄▄▄█████▓
    ▒ ▒ ▒ ▄▀░▒██▒  ██▒▓██▒▀█▀ ██▒▓█████▄ ▓██▒▓  ██▒ ▓▒
    ░ ▒ ▄▀▒░ ▒██░  ██▒▓██    ▓██░▒██▒ ▄██▒██▒▒ ▓██░ ▒░
      ▄▀▒   ░▒██   ██░▒██    ▒██ ▒██░█▀  ░██░░ ▓██▓ ░ 
    ▒███████▒░ ████▓▒░▒██▒   ░██▒░▓█  ▀█▓░██░  ▒██▒ ░ 
    ░▒▒ ▓░▒░▒░ ▒░▒░▒░ ░ ▒░   ░  ░░▒▓███▀▒░▓    ▒ ░░   
    ░░▒ ▒ ░ ▒  ░ ▒ ▒░ ░  ░      ░▒░▒   ░  ▒ ░    ░    
    ░ ░ ░ ░ ░░ ░ ░ ▒  ░      ░    ░    ░  ▒ ░  ░      
      ░ ░        ░ ░         ░    ░       ░       ░    
    ░                                  ░    ░                                                      

   Medical Dialogue Machine Translation (Chinese→Thai) - Colab HF Version
            Developer by JonusNattapong/zombit (Original Script)
         Powered by Hugging Face Helsinki-NLP Model
        CC BY-SA-NC 4.0 (Dataset) | https://github.com/JonusNattapong/MedMT
'''
    print(banner)

def auto_batch_size(n_samples):
    """Determine batch size for processing samples (not model inference batch)"""
    if n_samples >= 10000: return 200 # Reduced for potentially slower local processing
    elif n_samples >= 5000: return 100
    elif n_samples >= 1000: return 50
    elif n_samples >= 500: return 20
    elif n_samples >= 100: return 10
    else: return 5

def recommend_format(mode, output, n_samples):
    ext = ""
    if output is not None:
        ext = os.path.splitext(output)[1].lower()
    if ext in [".csv", ".json", ".jsonl", ".txt", ".arrow", ".parquet"]:
        return ext.lstrip(".")
    # Simplified recommendations for Colab
    if n_samples > 50000 and mode != "pretrain": return "parquet" # Parquet is good for large tabular data
    if mode == "pretrain": return "txt"
    return "csv" # Default to CSV for smaller/general cases

In [ ]:
# Cell 10: Argparse and Main function
def main_colab(args_dict):
    print_ascii_banner()
    
    # Convert dict to Namespace-like object for compatibility with original parser access
    class ArgsNamespace:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)
    args = ArgsNamespace(**args_dict)
    
    # Ensure translator is initialized (it's global, set in Cell 4)
    global translator
    if translator is None:
        print("[ERROR] Translator model could not be initialized. Please check Cell 4 output.")
        return
        
    recommended_format_val = recommend_format(args.mode, args.output, args.n_samples)
    if args.format is None:
        args.format = recommended_format_val
        print(f"[INFO] Auto-selected output format: {args.format.upper()} (recommended for mode '{args.mode}')")
    elif args.format != recommended_format_val:
        print(f"[INFO] You selected format '{args.format.upper()}'. Recommended format for mode '{args.mode}' is '{recommended_format_val.upper()}'.")

    # Adjust output path for Colab if it's not absolute
    output_file = args.output
    if output_file and not os.path.isabs(output_file) and not output_file.startswith("/content"):
        output_file = os.path.join("/content/datasets", os.path.basename(output_file) if os.path.basename(output_file) else unique_dataset_filename("dataset", "." + args.format, args.mode[0].upper()))
        print(f"[INFO] Adjusted output path for Colab: {output_file}")
    elif not output_file: # If output is None, generate a name in /content/datasets/
        datasets_dir = get_datasets_dir()
        base = args.mode + "_dataset"
        ext = "." + args.format
        output_file = os.path.join(datasets_dir, unique_dataset_filename(base, ext, args.mode[0].upper()))
        print(f"[INFO] Output file will be automatically named and saved to: {output_file}")
        
    args.output = output_file # Update args with potentially modified path
    
    print(f"[INFO] Mode: {args.mode} | Samples: {args.n_samples} | Output: {args.output} | Format: {args.format}")
    print(f"[INFO] Using Hugging Face model {MODEL_NAME} for translation.")
    print("[INFO] Generating dataset... (this may take a while for large datasets)" )
    time.sleep(0.5)

    generator_func = None
    if args.mode == 'dialogue':
        generator_func = generate_dialogue_dataset_hf
    elif args.mode == 'qa':
        generator_func = generate_qa_dataset_hf
    elif args.mode == 'reasoning':
        generator_func = generate_reasoning_dataset_hf
    elif args.mode == 'summarization':
        generator_func = generate_summarization_dataset_hf
    else:
        print(f"[ERROR] Unknown mode: {args.mode}")
        return

    # Batching for generation (not model inference batching, which is handled by pipeline or translate_text_hf)
    # This outer batching is to manage memory for large n_samples and allow periodic saving if implemented.
    # For now, it generates all 'n_samples' in one go by the generator function.
    # The original script's batching loop was more about calling the generator multiple times for smaller chunks.
    # Here, we call the generator once with the total n_samples.
    
    # The generator functions now take current_translator as an argument
    df_all = generator_func(n_samples=args.n_samples, seed=42, current_translator=translator) 
    
    if df_all.empty:
        print("[ERROR] No data generated. Please check logs and configurations.")
        return
        
    print(f"[INFO] Saving dataset to {args.output} as {args.format.upper()} ...")
    save_dataset(df_all, args.output, args.format)
    # print(f"[SUCCESS] Dataset saved. ({len(df_all)} samples)") # save_dataset already prints this
    print("[INFO] Dataset is licensed under CC BY-SA-NC 4.0 (https://creativecommons.org/licenses/by-nc-sa/4.0/)" )
    print(f"[INFO] Log file saved to: {LOG_FILE}")

In [ ]:
# Cell 11: Colab Parameter Setup and Execution Example

# --- CONFIGURE YOUR PARAMETERS HERE ---
colab_args = {
    "output": None,  # Set to a specific path like '/content/my_dataset.csv' or None for auto-naming in /content/datasets/
    "n_samples": 10,      # Number of samples to generate
    "mode": "dialogue",    # 'dialogue', 'qa', 'reasoning', 'summarization'
    "format": None       # 'csv', 'json', 'jsonl', 'txt', 'arrow'/'parquet', or None for auto-selection
}
# ---

# Ensure the translator is loaded before running main
if translator is None:
    print("[INFO] Translator was not loaded. Attempting to initialize now...")
    translator = initialize_translator() # Attempt to initialize if not already

if translator: # Proceed only if translator is available
    print(f"Executing with parameters: {colab_args}")
    main_colab(colab_args)
else:
    print("[FATAL ERROR] Translator could not be initialized. Cannot proceed with dataset generation. Check previous cell outputs for errors.")

# Example of generating a different type of dataset:
# colab_args_qa = {
#     "output": "/content/datasets/qa_colab_output.jsonl",
#     "n_samples": 5,
#     "mode": "qa",
#     "format": "jsonl"
# }
# if translator:
#     print(f"Executing QA generation with parameters: {colab_args_qa}")
#     main_colab(colab_args_qa)
# else:
#     print("[FATAL ERROR] Translator could not be initialized for QA example.")

## Notes on Usage and Potential Improvements:

1.  **Model Performance**: `Helsinki-NLP/opus-mt-zh-th` is a general-purpose Chinese-to-Thai translation model. Its performance on highly specialized medical terminology might vary. For production use, fine-tuning on a medical domain-specific corpus could improve results.
2.  **Processing Speed**: Translation, especially on CPU, can be slow for large datasets. The Hugging Face `pipeline` does some internal batching. For very large `n_samples`, consider running this on a Colab session with GPU acceleration (check `Runtime > Change runtime type`).
3.  **Error Handling**: The script includes basic error handling for translation failures, falling back to original English samples or error messages. Review the generated `LOG_FILE` for any issues.
4.  **Dataset Diversity**: The original script had mechanisms for prompt diversification for the DeepSeek LLM. For direct translation models, diversity mainly comes from the model's training. To increase diversity, one might consider:
    *   Using multiple different open-source translation models.
    *   Employing back-translation techniques (though this adds complexity).
5.  **Batching for Translation**: In `generate_dialogue_dataset_hf`, source texts are now batched before being sent to `translate_text_hf`. The `translate_text_hf` function itself uses the Hugging Face pipeline which can handle batches. You can adjust `batch_size` in `translate_text_hf` if needed, though the pipeline's default might be optimal.
6.  **Resource Limits**: Colab has usage limits. For very large datasets (e.g., >50k samples), you might need to run the script in multiple sessions or use a more robust environment.
7.  **Saving Output**: Datasets are saved in your Colab environment (typically under `/content/datasets/` if no specific path is given). Remember to download them before your Colab session ends, or mount Google Drive to save them persistently.
    ```python
    # To mount Google Drive (run in a new cell if needed)
    # from google.colab import drive
    # drive.mount('/content/drive')
    # Then you can set output paths like '/content/drive/MyDrive/my_dataset.csv'
    ```